In [1]:
#Conectarnos a la API de Twitter para recopilar datos.
#utilizar las relaciones tipo 'quién retuitea a quién' del topic Arduino

In [2]:
import tweepy
import pickle
from twitter_secrets import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify= True) #así espera y notifica

try:
    api.verify_credentials()
    print("Autentificación OK")
except tweepy.TweepError as e:
    print("Falló la autentificación")
    print(e)

Autentificación OK


In [3]:
#recopilación de tweets
#La API nos proporciona hasta 18000 tweets cada 15 minutos (100 tweets para cada una de las 180 peticiones máximo permitidas),

In [4]:
#paginación con objeto cursor: http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html
#cursor sirve para que no devuelvan tweets repetidos
#api.search(q='Trump',max_id=min(tweets_id)) - continua con la siguiente página

#guardo en formato pickle todos los tweets de arduino y en sqlite
import sqlite3
import pandas as pd

db = sqlite3.connect('tweets.sqlite')

for page_id, page in enumerate(tweepy.Cursor(
    api.search,q='arduino',count=100,
    tweet_mode='extended',
    languages = ['en', 'es']).pages()): # process status here
    
    fn = f'page_{page_id:06d}.pkl'
    with open("./pickle_arduino/"+fn, 'wb') as f:
        pickle.dump(page, f)
    
    #sqlite
    tweets = [str(i._json) for i in page]
    df = pd.DataFrame(tweets)
    df.to_sql("arduino",db,if_exists='append')

Rate limit reached. Sleeping for: 468


In [ ]:
#paginación con objeto cursor: http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html
import json
for page_id, page in enumerate(tweepy.Cursor(
    api.search,q='Trump',count=100,
    tweet_mode='extended',
    languages = ['en', 'es']).pages()): # process status here
    
    tweets = [i._json for i in page]
    fn = f'page_{page_id:06d}.json'
    with open("./json_trump/"+fn, 'w') as f:
        json.dump(tweets, f)
    
    #sqlite
    tweets_str = [str(i._json) for i in page]
    df = pd.DataFrame(tweets_str)
    df.to_sql("trump",db,if_exists='append')

In [6]:
#necesitaremos unos 100000 tweets
#en cada api.search me devuelve 100 tweets para las 180 permitidas cada 15 minutos
#al usar curso detiene en el último punto, si hiciera de nuevo api.search, repetiria tweets
#curso se debe usar si devuelve más del límite para que lo indexe

In [7]:
#obtener relaciones

In [7]:
from pathlib import Path
import pickle
import pandas as pd
import numpy as np

def read_pickled_pages(fn):
    pickle_files = Path('./pickle_arduino/').glob(fn)
    statuses = []
    for page_fn in pickle_files:
        #print(page_fn)
        with open(page_fn, 'rb') as f:
            statuses.extend(pickle.load(f))
    return statuses

def read_json_pages(fn):
    json_files = Path('./json_trump/').glob(fn)
    statuses = []
    for page_fn in json_files:
        #print(page_fn)
        with open(page_fn, 'rb') as f:
            statuses.extend(json.load(f))
    return statuses

def flatten_dict(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

def statuses_to_pandas(statuses):
    data = []
    for status in statuses:
        data.append(flatten_dict(status._json))
    return pd.DataFrame(data)

def statuses_to_pandas_json(statuses):
    data = []
    for status in statuses:
        data.append(flatten_dict(status))
    return pd.DataFrame(data)

def keep_only_user_id_fields(df):
    def is_user_id_column(c):
        return (('user_id' in c.lower() 
                 or ('user_mentions' in c.lower() and 'id' in c.lower()))
                 and not 'str' in c.lower())
    user_columns = sorted([c for c in df.columns if is_user_id_column(c)], key=len)
    df_nx = df[user_columns] 
    return df_nx


def user_df_to_graph(df_nx):

    graph = set()
    for index, row in df_nx.iterrows():
        user_id = row[0]
        assert(not np.isnan(user_id))
        for other_id in row[1:]:
            try:
                if other_id is not None and not np.isnan(other_id):
                    graph.add((int(user_id), int(other_id)))
            except TypeError:
                pass


    df_graph = pd.DataFrame(graph, columns=['src', 'dest'])
    return df_graph

statuses = read_pickled_pages('*.pkl') #lista con los statuses

In [8]:
len(statuses)

16269

In [9]:
tweets_pd = statuses_to_pandas(statuses)

In [10]:
tweets_pd.describe()

,id,display_text_range_0,display_text_range_1,entities_user_mentions_0_id,entities_user_mentions_0_indices_0,entities_user_mentions_0_indices_1,in_reply_to_status_id,in_reply_to_user_id,user_id,user_followers_count,...,quoted_status_entities_user_mentions_5_indices_0,quoted_status_entities_user_mentions_5_indices_1,extended_entities_media_0_additional_media_info_source_user_entities_description_urls_1_indices_0,extended_entities_media_0_additional_media_info_source_user_entities_description_urls_1_indices_1,quoted_status_entities_urls_2_indices_0,quoted_status_entities_urls_2_indices_1,retweeted_status_entities_symbols_0_indices_0,retweeted_status_entities_symbols_0_indices_1,user_entities_description_urls_4_indices_0,user_entities_description_urls_4_indices_1
count,1.626900e+04,16269.000000,16269.000000,1.097500e+04,10975.000000,10975.000000,1.644000e+03,1.700000e+03,1.626900e+04,1.626900e+04,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.266884e+18,2.962751,129.517364,2.343288e+17,7.363098,17.491845,1.266384e+18,3.406208e+17,4.450285e+17,4.384756e+04,...,152.0,165.0,119.0,142.0,78.0,101.0,89.0,93.0,172.0,195.0
std,8.270711e+14,27.548013,57.759935,4.310333e+17,21.582989,21.761236,6.501515e+15,4.983326e+17,5.219213e+17,8.604545e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.265530e+18,0.000000,1.000000,6.136000e+03,0.000000,4.000000,1.114963e+18,6.136000e+03,1.148100e+04,0.000000e+00,...,152.0,165.0,119.0,142.0,78.0,101.0,89.0,93.0,172.0,195.0
25%,1.266144e+18,0.000000,100.000000,3.388454e+07,3.000000,9.000000,1.266075e+18,1.549982e+08,2.929960e+08,9.500000e+01,...,152.0,165.0,119.0,142.0,78.0,101.0,89.0,93.0,172.0,195.0
50%,1.266836e+18,0.000000,139.000000,2.664008e+08,3.000000,13.000000,1.266822e+18,1.479397e+09,3.247039e+09,4.160000e+02,...,152.0,165.0,119.0,142.0,78.0,101.0,89.0,93.0,172.0,195.0
75%,1.267626e+18,0.000000,140.000000,4.760788e+09,3.000000,17.000000,1.267574e+18,9.176154e+17,1.000303e+18,1.470000e+03,...,152.0,165.0,119.0,142.0,78.0,101.0,89.0,93.0,172.0,195.0
max,1.268293e+18,658.000000,932.000000,1.268258e+18,270.000000,278.000000,1.268291e+18,1.268258e+18,1.268258e+18,1.882007e+07,...,152.0,165.0,119.0,142.0,78.0,101.0,89.0,93.0,172.0,195.0


In [11]:
#lista de relaciones id_user (user.id) y en retweeted_status.user.id

In [12]:
relaciones_pd=keep_only_user_id_fields(tweets_pd)

In [13]:
relaciones_pd.describe()

,user_id,in_reply_to_user_id,quoted_status_user_id,retweeted_status_user_id,entities_user_mentions_0_id,entities_user_mentions_1_id,entities_user_mentions_2_id,entities_user_mentions_3_id,entities_user_mentions_4_id,entities_user_mentions_5_id,...,retweeted_status_quoted_status_entities_user_mentions_0_id,retweeted_status_quoted_status_entities_user_mentions_1_id,retweeted_status_quoted_status_entities_user_mentions_2_id,retweeted_status_quoted_status_entities_user_mentions_3_id,extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_quoted_status_entities_media_0_source_user_id,retweeted_status_quoted_status_extended_entities_media_0_source_user_id,quoted_status_extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_quoted_status_extended_entities_media_0_additional_media_info_source_user_id
count,1.626900e+04,1.700000e+03,2.180000e+02,8.754000e+03,1.097500e+04,1.897000e+03,8.120000e+02,4.310000e+02,2.160000e+02,1.480000e+02,...,5.700000e+01,2.700000e+01,1.500000e+01,1.0,6.300000e+01,2.600000e+01,2.600000e+01,1.0,2.200000e+01,2.0
mean,4.450285e+17,3.406208e+17,2.359018e+17,2.277940e+17,2.343288e+17,1.899663e+17,2.051859e+17,1.408829e+17,2.189734e+17,1.037652e+17,...,2.741560e+17,5.222772e+17,4.136819e+08,952092048.0,5.197148e+17,1.204733e+17,1.204733e+17,266400754.0,1.731097e+08,266400754.0
std,5.219213e+17,4.983326e+17,4.461184e+17,4.239321e+17,4.310333e+17,3.999062e+17,4.198263e+17,3.546034e+17,4.448162e+17,2.922969e+17,...,4.890786e+17,5.556438e+17,1.036077e+09,NaN,5.446192e+17,3.436086e+17,3.436086e+17,NaN,1.147604e+08,0.0
min,1.148100e+04,6.136000e+03,8.166530e+05,1.148100e+04,6.136000e+03,5.279000e+03,7.415430e+05,1.334800e+04,6.296300e+04,6.296300e+04,...,6.761030e+05,9.003112e+06,1.361329e+07,952092048.0,3.331578e+07,2.168990e+07,2.168990e+07,266400754.0,3.835592e+07,266400754.0
25%,2.929960e+08,1.549982e+08,2.387986e+08,3.388454e+07,3.388454e+07,7.631604e+07,9.147862e+07,1.219762e+08,9.780696e+07,1.152518e+08,...,2.664008e+08,5.625622e+07,3.436468e+07,952092048.0,2.299427e+08,3.835592e+07,3.835592e+07,266400754.0,3.835592e+07,266400754.0
50%,3.247039e+09,1.479397e+09,2.664008e+08,2.664008e+08,2.664008e+08,2.664008e+08,2.664008e+08,3.514082e+08,3.729218e+08,2.664008e+08,...,3.026663e+08,2.670064e+09,2.664008e+08,952092048.0,4.529092e+09,3.835592e+07,3.835592e+07,266400754.0,2.664008e+08,266400754.0
75%,1.000303e+18,9.176154e+17,3.230520e+09,4.350227e+09,4.760788e+09,3.004269e+09,2.687794e+09,2.072317e+09,2.827404e+09,6.133241e+08,...,2.968165e+09,1.100588e+18,2.664008e+08,952092048.0,1.125153e+18,2.601159e+08,2.601159e+08,266400754.0,2.664008e+08,266400754.0
max,1.268258e+18,1.268258e+18,1.259268e+18,1.268179e+18,1.268258e+18,1.266584e+18,1.256350e+18,1.247259e+18,1.206979e+18,1.232407e+18,...,1.204510e+18,1.139636e+18,4.134255e+09,952092048.0,1.254139e+18,1.147923e+18,1.147923e+18,266400754.0,2.664008e+08,266400754.0


In [14]:
grafo = user_df_to_graph(relaciones_pd)

In [15]:
grafo

,src,dest
0,2933209888,33884545
1,24969935,101584084
2,225856617,10228272
3,2327559914,801472141099208704
4,1000303340209823744,1176339816838922240
...,...,...
13611,3153893019,265588785
13612,2797091621,15608410
13613,2616208171,70266297
13614,2444040222,2372055768


In [16]:
grafo.describe()

,src,dest
count,1.361600e+04,1.361600e+04
mean,3.750326e+17,1.967719e+17
std,5.062043e+17,4.068026e+17
min,1.148100e+04,3.380000e+02
25%,1.985729e+08,3.388454e+07
50%,1.958211e+09,2.664008e+08
75%,9.458984e+17,2.897286e+09
max,1.268258e+18,1.268258e+18


In [17]:
#guardo el grafo en formato csv
grafo.to_csv('grafo_arduino.csv', sep=' ', index=False, header=False)

In [18]:
statuses_json = read_json_pages('*.json') #lista con los statuses

In [19]:
len(statuses_json)

183489

In [20]:
tweets_json_pd = statuses_to_pandas_json(statuses_json)

In [21]:
tweets_json_pd.describe()

,id,display_text_range_0,display_text_range_1,entities_user_mentions_0_id,entities_user_mentions_0_indices_0,entities_user_mentions_0_indices_1,in_reply_to_status_id,in_reply_to_user_id,user_id,user_followers_count,...,retweeted_status_entities_urls_6_indices_0,retweeted_status_entities_urls_6_indices_1,retweeted_status_entities_urls_7_indices_0,retweeted_status_entities_urls_7_indices_1,retweeted_status_entities_urls_8_indices_0,retweeted_status_entities_urls_8_indices_1,retweeted_status_entities_urls_9_indices_0,retweeted_status_entities_urls_9_indices_1,retweeted_status_quoted_status_extended_entities_media_0_additional_media_info_source_user_entities_description_urls_0_indices_0,retweeted_status_quoted_status_extended_entities_media_0_additional_media_info_source_user_entities_description_urls_0_indices_1
count,1.834890e+05,183489.000000,183489.000000,1.690530e+05,169053.000000,169053.000000,2.131300e+04,2.200400e+04,1.834890e+05,1.834890e+05,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.268291e+18,3.408755,130.370191,2.364594e+17,3.916949,15.985667,1.267875e+18,3.199378e+17,4.243524e+17,6.148418e+03,...,178.0,201.0,203.0,226.0,228.0,251.0,253.0,276.0,53.0,76.0
std,3.407292e+12,21.493096,48.972632,4.321003e+17,13.011858,13.179005,1.017624e+16,4.853999e+17,5.156498e+17,1.851634e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.268285e+18,0.000000,5.000000,1.200000e+01,0.000000,2.000000,4.569749e+17,1.200000e+01,2.910000e+02,0.000000e+00,...,178.0,201.0,203.0,226.0,228.0,251.0,253.0,276.0,53.0,76.0
25%,1.268288e+18,0.000000,103.000000,3.580572e+07,3.000000,13.000000,1.268223e+18,3.691991e+07,3.379963e+08,9.700000e+01,...,178.0,201.0,203.0,226.0,228.0,251.0,253.0,276.0,53.0,76.0
50%,1.268291e+18,0.000000,140.000000,4.983529e+08,3.000000,15.000000,1.268278e+18,4.649087e+08,2.865137e+09,3.670000e+02,...,178.0,201.0,203.0,226.0,228.0,251.0,253.0,276.0,53.0,76.0
75%,1.268294e+18,0.000000,140.000000,4.105079e+09,3.000000,17.000000,1.268288e+18,8.229549e+17,9.610122e+17,1.443000e+03,...,178.0,201.0,203.0,226.0,228.0,251.0,253.0,276.0,53.0,76.0
max,1.268296e+18,696.000000,972.000000,1.268288e+18,290.000000,300.000000,1.268296e+18,1.268283e+18,1.268295e+18,4.846501e+07,...,178.0,201.0,203.0,226.0,228.0,251.0,253.0,276.0,53.0,76.0


In [22]:
relaciones_json_pd=keep_only_user_id_fields(tweets_json_pd)
relaciones_json_pd.describe()

,user_id,in_reply_to_user_id,quoted_status_user_id,retweeted_status_user_id,entities_user_mentions_0_id,entities_user_mentions_1_id,entities_user_mentions_2_id,entities_user_mentions_3_id,entities_user_mentions_4_id,entities_user_mentions_5_id,...,retweeted_status_quoted_status_entities_user_mentions_18_id,retweeted_status_quoted_status_entities_user_mentions_19_id,retweeted_status_quoted_status_entities_user_mentions_20_id,retweeted_status_quoted_status_entities_user_mentions_21_id,retweeted_status_quoted_status_entities_media_0_source_user_id,extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_quoted_status_extended_entities_media_0_source_user_id,quoted_status_extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_quoted_status_extended_entities_media_0_additional_media_info_source_user_id
count,1.834890e+05,2.200400e+04,3.774000e+03,1.452310e+05,1.690530e+05,2.645300e+04,1.039900e+04,4.426000e+03,2.292000e+03,1.381000e+03,...,3.000000e+00,3.000000e+00,2.000000e+00,1.0,1.371000e+03,1.927000e+03,1.371000e+03,1.500000e+01,1.500000e+02,1.500000e+01
mean,4.243524e+17,3.199378e+17,2.102778e+17,2.277447e+17,2.364594e+17,2.370091e+17,2.584985e+17,2.756973e+17,2.914849e+17,3.199255e+17,...,8.233300e+17,8.360182e+08,8.180195e+17,42760278.0,8.663079e+16,5.404318e+17,8.663079e+16,3.235788e+17,2.991766e+17,1.343275e+17
std,5.156498e+17,4.853999e+17,4.207066e+17,4.248422e+17,4.321003e+17,4.311516e+17,4.438379e+17,4.569867e+17,4.669129e+17,4.758295e+17,...,7.130320e+17,1.398523e+09,1.357065e+17,NaN,2.866295e+17,5.309487e+17,2.866295e+17,4.847296e+17,4.746441e+17,3.557742e+17
min,2.910000e+02,1.200000e+01,2.461030e+05,7.670000e+02,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,...,2.200545e+09,1.724391e+07,7.220605e+17,42760278.0,6.210300e+04,3.968000e+03,6.210300e+04,2.507388e+07,1.652541e+06,1.663528e+07
25%,3.379963e+08,3.691991e+07,2.560549e+07,3.997997e+07,3.580572e+07,2.507388e+07,2.507388e+07,2.507388e+07,2.507388e+07,2.560549e+07,...,6.158853e+17,2.860713e+07,7.700400e+17,42760278.0,2.507388e+07,2.664201e+07,2.507388e+07,4.640677e+08,2.955751e+08,2.507388e+07
50%,2.865137e+09,4.649087e+08,2.364879e+08,5.429630e+08,4.983529e+08,1.862770e+08,2.175432e+08,2.929293e+08,4.119299e+08,6.127826e+08,...,1.231771e+18,3.997035e+07,8.180195e+17,42760278.0,3.306310e+08,8.438671e+17,3.306310e+08,2.800581e+09,5.513498e+08,2.507388e+07
75%,9.610122e+17,8.229549e+17,2.917052e+09,3.667004e+09,4.105079e+09,3.701542e+09,7.536932e+17,7.498039e+17,8.188972e+17,8.275901e+17,...,1.234995e+18,1.245405e+09,8.659990e+17,42760278.0,4.606184e+08,1.003160e+18,4.606184e+08,7.653545e+17,7.138393e+17,1.156823e+08
max,1.268295e+18,1.268283e+18,1.268281e+18,1.268288e+18,1.268288e+18,1.268279e+18,1.268059e+18,1.268242e+18,1.267573e+18,1.267750e+18,...,1.238219e+18,2.450840e+09,9.139785e+17,42760278.0,1.267593e+18,1.268258e+18,1.267593e+18,1.148213e+18,1.243560e+18,1.087380e+18


In [23]:
grafo_json = user_df_to_graph(relaciones_json_pd)
grafo_json.describe()
grafo_json.to_csv('grafo_json_trump.csv', sep=' ', index=False, header=False)

In [24]:
len(grafo_json)

246622

In [25]:
len(grafo)

13616

In [26]:
grafo

,src,dest
0,2933209888,33884545
1,24969935,101584084
2,225856617,10228272
3,2327559914,801472141099208704
4,1000303340209823744,1176339816838922240
...,...,...
13611,3153893019,265588785
13612,2797091621,15608410
13613,2616208171,70266297
13614,2444040222,2372055768


In [27]:
grafo_json

,src,dest
0,1146565767686701056,73194395
1,1082284429026967552,73194395
2,356438982,55968000
3,992167212,68498034
4,3057818228,3300864488
...,...,...
246617,2381303568,60751951
246618,2519941281,1385878826
246619,1246847906612752384,79379491
246620,155936909,1047598652510937088


In [57]:
#Parte 2 Analizar la red

In [58]:
#Limpieza de datos

In [59]:
#Análisis de la red

In [60]:
#Análisis de los nodos

In [61]:
#Detección de comunidades

In [ ]:



'''
#usar la centralidad de katz - para medir la centralidad 
nx.katz_centrallity
nx.katz_centrallity_numpy (usa el algoritmo de numpy)
nx.eigenvector_centrallity (alternativa)
nx.eigenvector_centrallity_numpy (alternativa)
nx.pagerank (alternativa)
'''  
        
#el objetivo es ver comunidades de los grafos
#se ven dos comunidades al rededor el partido democrata y republicano
#intermediación - conexiones (es muy grande porque son los usuarios a los que siguen los de in partido y otro)
#autoridad - la cuenta de Trump tiene mucha y la de Joe Biden no tanto
          
#dependiendo de lo que recortemos la red aparecerán más comunidades pequeñas. Depende de l grado de relacción que establezcamos
# un grado mínimo es 2 que es que tienen dos relaciones (explica)
          
#es un análisis entrno a una instantanea en concreto

#como mandamos el proyecto y transparencias del último día

#en ejercicio 1 si tarda mucho en hacer el grafo y pasar al siguiente y simplificar la red, pude tardar entre 10 y 20 minutos

#Mandar el notebook y el fichero del grafo y el fichero de datos